---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    # Your code here
    result1 = df.str.extract('(?P<raw>(?P<month>0?[1-9]|1[0-2])[/](?:(?P<day>0?[1-9]|1[0-9]|2[0-9]|3[01])[/])?(?P<year>\d{2,4}))')
    result2 = df.str.extract('(?P<raw>(?P<day>\d{1,2}) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[.,]? (?P<year>\d{2,4}))')
    result3 = df.str.extract('(?P<raw>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[.,]? (?P<day>\d{1,2})[,]? (?P<year>\d{2,4}))')
    result4 = df.str.extract('(?P<raw>(?P<month>0?[1-9]|1[0-2])[-](?P<day>0?[1-9]|1[0-9]|2[0-9]|3[01])[-](?P<year>\d{2,4}))')
    
    result1 = result1.fillna('')
    result2 = result2.fillna('')
    result3 = result3.fillna('')
    result4 = result4.fillna('')
    

    result = pd.DataFrame(columns=['raw', 'month', 'day', 'year'])
    for name in result:
        result[name] = result1[name] + result2[name] + result3[name] + result4[name]
        
    result6 = df[result['raw']==''].str.extract('(?P<raw>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[.,]? (?P<year>\d{2,4}))')
    for i in result6.index:
        result.raw[i] = result6.raw[i]
        result.month[i] = result6.month[i]
        result.year[i] = result6.year[i]
    result = result.fillna('')
    
    result5 = df[result['raw']==''].str.extract('(?P<raw>(?P<year>\d{4}))')
    for i in result5.index:
        result.raw[i] = result5.raw[i]
        result.year[i] = result5.year[i]
    result = result.fillna('')
    
    result['year'] = result['year'].apply(lambda x: x if len(x)==4 else '19'+x)
    month_dict = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
                  'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12',
                  'January': '1', 'February': '2', 'March': '3', 'April': '4', 'June': '6',
                  'July': '7', 'August': '8', 'September': '9', 'October': '10', 'November': '11',
                  'December': '12', '': '1',
                  'Decemeber': '12', 'Janaury': '1'}
    result['month'] = result['month'].apply(lambda x: month_dict[x] if x in month_dict else x)
    result['day'] = result['day'].apply(lambda x: '1' if x=='' else x)
    
    for name in ['year', 'month', 'day']:
        result[name] = result[name].astype('int')
    
    result = result.sort_values(['year', 'month', 'day'])

#     return result[result['raw']=='']
    return pd.Series(result.index)

date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this w

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [3]:
df[271]

'.Spoke to sister Naomi Ely 708-810-7787 who reports he has been doing much better since he went to Dysart Clinic (he was drinking for a month leading up to this, his ammonia was high, and physicians were worried about early).  She feels his cognition is back to baseline, "100% better".  She says he has been successful in abstaining from substances as far as she knows, thinks a schedule is useful to him, doctor\'s appts etc.  Notes that he returned from LA in August 2008, gets bouts of "exhaustion" even in sobriety.  She denies ever witnessing any periods of manic behavior from patient.  Their father has dementia that started at age 84.  Notes patient is living with uncle in Black River Falls (uncle is 89), lived with sister 3 months who also takes care of her own father in Talladega.  She knows he is working on getting social security, subsidizing housing.  Stable situation with patient\'s girlfriend Nutt.Suicidal Behavior Hx of Suicidal Behavior: No\n'